In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00



##**Importing Necessary Libraries**##



In [2]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SGConv
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [3]:
torch.__version__

'2.4.1+cu121'

In [4]:
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00


##**Dataset Preparation (Cora)**##

In [5]:
path = "dataset"  # Path to download the dataset
dataset = Planetoid(path, "Cora")  # Downloads and loads the Cora dataset
data = dataset[0]  # Get the first (and only) graph from the dataset

Processing...
Done!


In [6]:
print('Cora:', data)  # Print the structure of the Cora data object

Cora: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


##**Model Construction (SGConv)**##

In [7]:
SGC_model = SGConv(in_channels= data.num_features, # Number of features in each node (i.e., input dimension).
                   out_channels= dataset.num_classes, # The number of output classes (i.e., output dimension).
                   K = 1, # Number of hops or layers. Setting K=1 means one step aggregation from neighboring nodes.
                   cached =True) # Cache the computation of graph convolutions during training to speed up the process.
'''
SGConv is a lightweight version of GCN, with the idea that the feature aggregation step (graph convolution)
can be simplified by removing non-linearity and stacking layers.
'''

'\nSGConv is a lightweight version of GCN, with the idea that the feature aggregation step (graph convolution)\ncan be simplified by removing non-linearity and stacking layers.\n'

In [8]:
# GET EMBEDDING
print(" Shape of the original data: ", data.x.shape)
print(" Shape of the embedding data: ", SGC_model(data.x,data.edge_index).shape)

 Shape of the original data:  torch.Size([2708, 1433])
 Shape of the embedding data:  torch.Size([2708, 7])


In [11]:
SGC_model(data.x,data.edge_index)[0]

tensor([-0.0031, -0.0287,  0.0033,  0.0818, -0.0011,  0.0102,  0.0392],
       grad_fn=<SelectBackward0>)

In [17]:
F.softmax(SGC_model(data.x,data.edge_index), dim=1)[0]

tensor([0.1403, 0.1367, 0.1412, 0.1527, 0.1406, 0.1422, 0.1463],
       grad_fn=<SelectBackward0>)

In [18]:
F.softmax(SGC_model(data.x,data.edge_index), dim=1)[0].sum()

tensor(1.0000, grad_fn=<SumBackward0>)

In [19]:
F.log_softmax(SGC_model(data.x,data.edge_index), dim=1)[0]

tensor([-1.9641, -1.9897, -1.9577, -1.8791, -1.9621, -1.9508, -1.9218],
       grad_fn=<SelectBackward0>)

In [21]:
F.log_softmax(SGC_model(data.x,data.edge_index), dim=1)[0].sum()

tensor(-13.6253, grad_fn=<SumBackward0>)

##**SGC network architecture for node classification**##

In [22]:
class SGCNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SGConv(in_channels= data.num_features,
                            out_channels= dataset.num_classes,
                            K = 1, cached =True)

    def forward(self):
        x = self.conv1(data.x, data.edge_index)
        return F.log_softmax(x, dim=1) # the output is a probability distribution over different classes for each node.

##**Training Function**##

In [23]:
# Optimizer Setup
optimizer = torch.optim.Adam(SGC_model.parameters(), # to update the model parameters
                             lr=0.2, # Learning rate
                             weight_decay=0.005) # L2 regularization to prevent overfitting

In [28]:
# What are the learning parameters:
for i, parameter in SGC_model.named_parameters():
    print("Parameter: {}".format(i))
    print("Shape: ",parameter.shape)

Parameter: lin.weight
Shape:  torch.Size([7, 1433])
Parameter: lin.bias
Shape:  torch.Size([7])


In [29]:
def train():
    SGC_model.train() # Set the model to training mode
    optimizer.zero_grad()  # Reset gradients to avoid accumulation of gradients from previous updates
    predicted_y = SGC_model()  # Get the predicted log-probabilities for the nodes in the graph.
    true_y = data.y  # True labels
    losses = F.nll_loss(predicted_y[data.train_mask], true_y[data.train_mask])  # Negative Log Likelihood Loss
    '''
    This loss function is used for multi-class classification.
    It compares the predicted probabilities (predicted_y) with the true labels (true_y),
    but only for nodes in the training set (data.train_mask).
    '''
    losses.backward()  # Compute the gradients
    optimizer.step()  # Update the model's weights (parameters)

##**Testing Function**##

In [30]:
def test():
    SGC_model.eval()  # Set model to evaluation mode (This turns off dropout, batch normalization, and other training-only behaviors)
    logits = SGC_model()  # a forward pass through the model to obtain the log-probabilities (also called logits) for all nodes in the dataset.
    '''
    The output logits is a matrix where each row corresponds to a node, and each column corresponds to a class.
    Each element represents the log-probability that a node belongs to a particular class.
    '''
    accs = []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]  # Get predicted labels (index of max log-probability)
        '''
        max(1) finds the maximum log-probability along the columns (which correspond to the different classes).
        [1] selects the index of the class with the highest log-probability, i.e., the predicted class label for each node.
        '''
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()  # Compute accuracy
        '''
        pred.eq(data.y[mask]): This compares the predicted class labels (pred) with the true class labels (data.y[mask])
        for the nodes in the current mask (either training, validation, or test). This results in a tensor of booleans,
        where True indicates a correct prediction, and False indicates an incorrect prediction.

        .sum().item(): Counts the number of correct predictions by summing the True values.

        mask.sum().item(): Counts the total number of nodes in the current subset (either training, validation, or test).

        acc = correct_predictions / total_nodes: Computes the accuracy as the proportion of correctly predicted nodes.
        '''
        accs.append(acc)
    return accs

##**Training and Evaluation Loop**##

In [31]:
# Check if GPU (CUDA) is available, otherwise use CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Prints the selected device ("cuda" or "cpu").
print('Device: {}'.format(device))

Device: cpu


In [32]:
SGC_model, data = SGCNet().to(device), data.to(device)

In [35]:
best_val_acc = test_acc = 0
for epoch in range(1, 101):
    train()  # Train the model (the train() function is called to update the model parameters)
    train_acc, val_acc, tmp_test_acc = test()  # Test the model (The test() function is called to evaluate the model’s performance on the training, validation, and test sets)
    if val_acc > best_val_acc:  # If the validation accuracy is the best so far, update it
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))

Epoch: 001, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 002, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 003, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 004, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 005, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 006, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 007, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 008, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 009, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 010, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 011, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 012, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 013, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 014, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 015, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 016, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 017, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 018, Train: 0.1571, Val: 0.1680, Test: 0.1680
Epoch: 019, Train: 0.1571, Val: 0.1680, Test: 